In [ ]:
# Import các thư viện cần thiết
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Thiết lập hiển thị cho matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

# Thiết lập hiển thị cho pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)

print("Đã thiết lập môi trường")


In [ ]:
# Import các module từ trainer
from trainer.main import init_trainer, prepare_for_student

# Sử dụng CUDA nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Tìm kiếm model
model_path = "dcrkt_model_fold_0.pt"
if not os.path.exists(model_path):
    print(f"Không tìm thấy file mô hình: {model_path}")
    print("Tìm kiếm mô hình trong thư mục checkpoints...")
    model_path = "checkpoints/dcrkt_model_fold_0.pt"
    if not os.path.exists(model_path):
        print(f"Không tìm thấy file mô hình: {model_path}")
        model_path = None

# Khởi tạo hệ thống
print("Đang khởi tạo hệ thống Knowledge Map Trainer...")
model, dashboard, dfs, mappings = init_trainer(model_path, device=device)
print("Đã khởi tạo hoàn tất!")


In [ ]:
# Xem thông tin về học sinh
student_ids = list(mappings["user2idx"].keys())
print(f"Số lượng học sinh: {len(student_ids)}")
print(f"Ví dụ ID học sinh: {student_ids[:5]}")

# Xem thông tin về câu hỏi
question_ids = list(mappings["question2idx"].keys())
print(f"\nSố lượng câu hỏi: {len(question_ids)}")
print(f"Ví dụ ID câu hỏi: {question_ids[:5]}")

# Xem thông tin về concept
concept_ids = list(mappings["kc2idx"].keys())
print(f"\nSố lượng concept: {len(concept_ids)}")
print(f"Ví dụ ID concept: {concept_ids[:5]}")

# Hiển thị vài dòng dữ liệu tương tác
print("\nMẫu dữ liệu tương tác:")
display(dfs["interaction_df"].head())


In [ ]:
# Chọn một học sinh để trực quan hóa
student_id = student_ids[0]  # Chọn học sinh đầu tiên trong danh sách
print(f"Đã chọn học sinh ID: {student_id}")

# Chuẩn bị mô hình cho học sinh
print(f"Đang tải dữ liệu lịch sử cho học sinh {student_id}...")
model = prepare_for_student(model, student_id, dfs, mappings)
print("Đã tải xong dữ liệu lịch sử!")


In [ ]:
# Hiển thị bản đồ kiến thức
knowledge_map = dashboard.display_knowledge_map(student_id, save=True)
plt.show()


In [ ]:
# Hiển thị heatmap mức độ thành thạo
heatmap = dashboard.display_mastery_heatmap(student_id, save=True)
plt.show()


In [ ]:
# Tạo kế hoạch ôn tập Active Recall
print("Kế hoạch ôn tập Active Recall cho 7 ngày tới:")
dashboard.display_study_plan(student_id, plan_type="active_recall", days=7)


In [ ]:
# Tạo kế hoạch ôn tập Spaced Repetition
print("Kế hoạch ôn tập Spaced Repetition:")
dashboard.display_study_plan(student_id, plan_type="spaced_repetition", days=30)


In [ ]:
# Chọn một câu hỏi để dự đoán
question_id = question_ids[0]  # Câu hỏi đầu tiên

# Dự đoán kết quả
print(f"Dự đoán kết quả cho học sinh {student_id} và câu hỏi {question_id}:")
prediction = dashboard.display_prediction(student_id, question_id, save=True)
plt.show()


In [ ]:
# Chọn 5 câu hỏi tiếp theo để mô phỏng
future_questions = question_ids[1:6]

# Mô phỏng quá trình học
print(f"Mô phỏng quá trình học cho học sinh {student_id} với 5 câu hỏi:")
simulation = dashboard.simulate_learning_path(student_id, future_questions)
plt.show()


In [ ]:
# Tạo dashboard tương tác
try:
    from IPython.display import display
    print("Hiển thị dashboard tương tác:")
    display(dashboard.interactive_dashboard())
except Exception as e:
    print(f"Không thể hiển thị dashboard tương tác: {e}")
    print("Đảm bảo rằng bạn đã cài đặt ipywidgets và đang chạy trong Jupyter Notebook.")
